In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def make_values_continuous(nnu_vals):
    if len(nnu_vals.shape) == 2:
        (N1, N2) = nnu_vals.shape

        for k in range(N2):
            for i in range(1, N1):
                diff = nnu_vals[i, k] - nnu_vals[i - 1, k]
                if abs(diff) > 0.5:
                    nnu_vals[i, k] -= 1 * np.sign(diff)
    else:
        N1 = len(nnu_vals)
        for i in range(1, N1):
            diff = nnu_vals[i] - nnu_vals[i - 1]
            if abs(diff) > 0.5:
                nnu_vals[i] -= 1 * np.sign(diff)

    return nnu_vals

In [ ]:
s0 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])

In [ ]:
kx_sym, ky_sym = sp.symbols('k_x k_y', real = True)
ksymbols = [kx_sym, ky_sym]
alpha = sp.symbols('alpha', real = True, positive = True)
gamma_z, lambda_z = sp.symbols('gamma_z lambda_z', real = True)

In [ ]:
hrtp = sp.sin(2*kx_sym) * sx
hrtp += sp.sin(kx_sym) * sp.sin(ky_sym) * sy
hrtp += - (alpha + sp.cos(2*kx_sym) + sp.cos(ky_sym)) * sz 

In [ ]:
Nx = 101
Ny = 101
Nbands = 2
Nocc = 1

params = {}
params["Nx"] = Nx
params["Ny"] = Ny
params["Nz"] = 0
params["Nbands"] = Nbands
params["Nocc"] = Nocc

In [ ]:
Kxs = np.linspace(0, 2*np.pi, Nx)
Kys = np.linspace(0, 2*np.pi, Ny)

In [ ]:
H_fixparam = hrtp.subs({alpha : 1})
hfunc = sp.lambdify((kx_sym,ky_sym), H_fixparam, modules = "numpy")

In [ ]:
eigenvalues = np.zeros((Nx,Ny,Nbands))
eigenvectors = np.zeros((Nx,Ny,Nbands,Nbands), dtype = complex)

for i in range(Nx):
    for j in range(Ny):
        vals, vecs = np.linalg.eigh(hfunc(Kxs[i], Kys[j]))
        
        ind = np.argsort(vals)
        eigenvalues[i,j,:] = vals[ind]
        eigenvectors[i,j,:,:] = vecs[:,ind]

In [ ]:
links_y_occ = np.zeros((Nx,Ny), dtype = np.complex128)
links_y_unocc = np.zeros((Nx,Ny), dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        ol_occ = np.dot(eigenvectors[i,(j+1)%Ny,:,0].conj().T, eigenvectors[i,j,:,0])
        ol_unocc = np.dot(eigenvectors[i,(j+1)%Ny,:,1].conj().T, eigenvectors[i,j,:,1])

        links_y_occ[i,j] = ol_occ/np.abs(ol_occ)
        links_y_unocc[i,j] = ol_unocc/np.abs(ol_unocc)

wloops_occ = np.ones(Nx, dtype = np.complex128)
wloops_unocc = np.ones(Nx, dtype = np.complex128)

for i in range(Nx):
    for j in range(Ny):
        wloops_occ[i] *= links_y_occ[i,j]
        wloops_unocc[i] *= links_y_unocc[i,j]

In [ ]:
phase_occ = np.angle(wloops_occ)/(2*np.pi)
phase_unocc = np.angle(wloops_unocc)/(2*np.pi)

phase_occ = make_values_continuous(phase_occ)
phase_unocc = make_values_continuous(phase_unocc)

In [ ]:
fig, axs = plt.subplots(1,1, figsize = (4,3))
plt.subplots_adjust(bottom=0.1, right=1)
plt.subplots_adjust(wspace=0.5)

plt.plot(Kxs,phase_occ, label = "occ", lw = 4)
plt.plot(Kxs,phase_unocc, label = "unocc", lw = 4)

plt.xlabel(r"$k_x$", fontsize = 16, labelpad = -1)
plt.ylabel(r"$\nu$", fontsize = 16, labelpad = -5)
plt.xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi], [r"$0$", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize = 14)
plt.yticks(fontsize = 14)
plt.legend(fontsize = 10, loc = "lower right")
plt.tight_layout()
plt.show()

In [ ]:
def get_U1Gauge(states):
    (Nx, Ny, _) = states.shape

    U1_gauges = np.zeros(Nx*Ny*2).reshape((Nx,Ny,2)).astype(np.complex128)

    for ii in range(Nx):
        for jj in range(Ny):
            dotprod1 = np.dot(np.conj(states[(ii+1)%Nx,jj,:].T), states[ii,jj,:])
            dotprod2 = np.dot(np.conj(states[ii,(jj+1)%Ny,:].T), states[ii,jj,:])

            U1_gauges[ii,jj,0] = dotprod1/np.abs(dotprod1)
            U1_gauges[ii,jj,1] = dotprod2/np.abs(dotprod2)

    return U1_gauges

def get_numerical_BerryCurvature(states):
    (Nx, Ny, _) = states.shape

    U1_gauges = get_U1Gauge(states)
    
    BerryCurvature = np.zeros(Nx*Ny).reshape((Nx,Ny)).astype(np.complex128)

    for ii in range(Nx):
        for jj in range(Ny):
            BerryCurvature[ii,jj]  = np.log(U1_gauges[ii,jj,0] 
                                            * U1_gauges[(ii+1)%Nx,jj,1] 
                                            / U1_gauges[ii,(jj+1)%Ny,0] 
                                            / U1_gauges[ii,jj,1])
            
    return BerryCurvature

def get_ChernNumber(BerryCurvature, X = None, Y = None):

    (Nx, Ny) = BerryCurvature.shape

    if X == None:
        x1 = 0
        x2 = Nx
    else:
        x1, x2 = X

    if Y == None:
        y1 = 0
        y2 = Ny
    else:
        y1, y2 = Y

    ChernNumber = np.sum(BerryCurvature[x1:x2,y1:y2]) / (2 * np.pi * 1.j)

    return ChernNumber

In [ ]:
BerryCurvature = get_numerical_BerryCurvature(eigenvectors[:,:,:,0])
ChernNumber = get_ChernNumber(BerryCurvature[:Nx//2,:])
print("Chern number: ", ChernNumber)